### Final Project, due Tuesday May 16th by Noon.

---

**Instructions:**

- Submit your answers as a .DATA302 file
- The database your code is checked on **will not contain any views** initially, so make sure to include code to create any views you need under the 'setup' key in your submission.
- Remember to use the CheckMySubmission() tool to validate spelling of key names, etc. before submitting.
- Thanks for a great semester!

>_May you do good and not evil<br>
>May you find forgiveness for yourself and forgive others<br>
>May you share freely, never taking more than you give._<br><br>
>-Richard Hipp

**Data Description:**
The dataset used for this project is a hypothetical record of a hardware store. It is organized into separate relational databases in third normal form. It includes sales on specific types of products with profit information as well as information on customers such as their names and place of residence and dates of purchase. The SQL commands are answering questions that test advanced database manipulation and information gathering techniques.

In [3]:
import store as s
import pandas as pd
import autog_student as ag

import json
from collections import defaultdict as dd

In [4]:
from importlib import reload
reload(s);

---
1) Generate a summary, by month and year of how our store is performing.

Have your query return the following:
 - year_month (as a single column, like YYYY-MM)
 - Sales: total sales for the month (i.e., sum of qty * unit price)
 - NumOrders: number of orders placed for the month
 - NumCust: number of _distinct_ customers who made a purchase (i.e. only count the customer at most once per month)
 - OrdersPerCust: average number of orders per customer (i.e. NumOrders/NumCust)
 - SalesPerCust: average sales per customer (i.e. Sales/NumCust)
 - SalesPerOrder: average sales per order (i.e. Sales/NumOrders)

Sort the results should by year_month, in ascending order.

_Hint: Watch out for integer division!_

In [5]:
sql1 = """
WITH year_month as
(
SELECT date,
    substr(date, 1, 7) as year_month
FROM tOrder
)
SELECT year_month, Sales, NumOrders, NumCust, 
        (NumOrders / (NumCust * 1.0)) as OrdersPerCust, 
        (Sales / NumCust) as SalesPerCust, 
        (Sales / NumOrders) as SalesPerOrder
FROM(
    SELECT *, sum(tot) as Sales, count(DISTINCT(order_id)) as NumOrders, count(DISTINCT cust_id) as NumCust
    FROM(
        SELECT DISTINCT(date), *, (qty * unit_price) as tot
        FROM year_month
        JOIN tOrder USING (date)
        JOIN tOrderDetail USING (order_id)
        JOIN tProd USING (prod_id)
        GROUP BY order_id, prod_id
    )
    GROUP BY year_month
)
ORDER BY year_month ASC
;"""
s.RunQuery(sql1)

,year_month,Sales,NumOrders,NumCust,OrdersPerCust,SalesPerCust,SalesPerOrder
0,2019-01,68464.61,91,85,1.070588,805.466000,752.358352
1,2019-02,55560.32,80,73,1.095890,761.100274,694.504000
2,2019-03,19191.68,51,49,1.040816,391.666939,376.307451
3,2019-04,20912.07,48,46,1.043478,454.610217,435.668125
4,2019-05,11973.34,50,46,1.086957,260.290000,239.466800
5,2019-06,13737.30,43,41,1.048780,335.056098,319.472093
6,2019-07,22095.05,45,41,1.097561,538.903659,491.001111
7,2019-08,15675.05,51,49,1.040816,319.898980,307.353922
8,2019-09,9360.38,40,39,1.025641,240.009744,234.009500
9,2019-10,48411.35,58,51,1.137255,949.242157,834.678448


---
2) In which month did we have the lowest total sales?

Return one record with:
- year_month (as a single column, with values like YYYY-MM)
- sales (sum of qty * unit_price)

In [6]:
sql2 = """
WITH year_month as
(
SELECT date,
    substr(date, 1, 7) as year_month
FROM tOrder
)
SELECT year_month, Sales
FROM(
    SELECT year_month, Sales, NumOrders, NumCust, 
            (NumOrders / NumCust) as OrdersPerCust, 
            (Sales / NumCust) as SalesPerCust, 
            (Sales / NumOrders) as SalesPerOrder
    FROM(
        SELECT *, sum(tot) as Sales, count(DISTINCT(order_id)) as NumOrders, count(DISTINCT cust_id) as NumCust
        FROM(
            SELECT DISTINCT(date), *, (qty * unit_price) as tot
            FROM year_month
            JOIN tOrder USING (date)
            JOIN tOrderDetail USING (order_id)
            JOIN tProd USING (prod_id)
            GROUP BY order_id, prod_id
        )
        GROUP BY year_month
    )
)
ORDER BY Sales ASC
LIMIT 1
;"""
s.RunQuery(sql2)

,year_month,Sales
0,2019-09,9360.38


---

3. In the month determined from the previous question, generate a list of our total sales by state.  Make sure that all states are included, even if they have no sales (50 states + PR and DC = 52 total records).

Return:

- The two-letter state abbreviation (st)
- Total sales for the month in question

Order the results by the state abbreviation, ascending

In [7]:
sql3 = """
WITH year_month as
(
SELECT date,
    substr(date, 1, 7) as year_month
FROM tOrder
),
SalesPerState as
(
    SELECT st, sum(Sales) as TotSales
    FROM (   
        SELECT DISTINCT(date), (unit_price * qty) as Sales, *
        FROM tCust
        JOIN tOrder USING (cust_id)
        JOIN tOrderDetail USING (order_id)
        JOIN tProd USING (prod_id)
        JOIN tZip USING (zip)
        JOIN year_month USING (date)
        WHERE year_month IN (
        SELECT year_month
        FROM(
            SELECT year_month, Sales, NumOrders, NumCust, 
                    (NumOrders / NumCust) as OrdersPerCust, 
                    (Sales / NumCust) as SalesPerCust, 
                    (Sales / NumOrders) as SalesPerOrder
            FROM(
                SELECT *, sum(tot) as Sales, count(DISTINCT(order_id)) as NumOrders, count(DISTINCT cust_id) as NumCust
                FROM(
                    SELECT DISTINCT(date), *, (qty * unit_price) as tot
                    FROM year_month
                    JOIN tOrder USING (date)
                    JOIN tOrderDetail USING (order_id)
                    JOIN tProd USING (prod_id)
                    GROUP BY order_id, prod_id
                )
                GROUP BY year_month
            )
        )
        ORDER BY Sales ASC
        LIMIT 1
        )
    )
    GROUP BY st
)
SELECT st, IFNULL(TotSales, 0) as TotalSales
FROM(
    SELECT st, IFNULL(0.0, 0) as TotalSales
    FROM tZip
    GROUP BY st
)
LEFT JOIN SalesPerState USING (st)
;"""
s.RunQuery(sql3)

,st,TotalSales
0,AK,0.00
1,AL,28.00
2,AR,0.00
3,AZ,0.00
4,CA,557.00
5,CO,0.00
6,CT,0.00
7,DC,0.00
8,DE,0.00
9,FL,414.00


---

4. For the list of states above that had $0 sales, generate a list of all the customers in those states, along with how much they have bought from us since then.

Return:
- cust_id
- first
- last
- addr
- city
- st
- zip
- the customer's total sales from all months after the month from question 2

Order the results by cust_id, ascending

In [8]:
sql4 = """
WITH year_month as
(
SELECT date,
    substr(date, 1, 7) as year_month
FROM tOrder
),
SalesPerState as
(
    SELECT st, sum(Sales) as TotSales
    FROM (   
        SELECT DISTINCT(date), (unit_price * qty) as Sales, *
        FROM tCust
        JOIN tOrder USING (cust_id)
        JOIN tOrderDetail USING (order_id)
        JOIN tProd USING (prod_id)
        JOIN tZip USING (zip)
        JOIN year_month USING (date)
        WHERE year_month IN (
        SELECT year_month
        FROM(
            SELECT year_month, Sales, NumOrders, NumCust, 
                    (NumOrders / NumCust) as OrdersPerCust, 
                    (Sales / NumCust) as SalesPerCust, 
                    (Sales / NumOrders) as SalesPerOrder
            FROM(
                SELECT *, sum(tot) as Sales, count(DISTINCT(order_id)) as NumOrders, count(DISTINCT cust_id) as NumCust
                FROM(
                    SELECT DISTINCT(date), *, (qty * unit_price) as tot
                    FROM year_month
                    JOIN tOrder USING (date)
                    JOIN tOrderDetail USING (order_id)
                    JOIN tProd USING (prod_id)
                    GROUP BY order_id, prod_id
                )
                GROUP BY year_month
            )
        )
        ORDER BY Sales ASC
        LIMIT 1
        )
    )
    GROUP BY st
),
StatesSales as(
    SELECT st, IFNULL(TotSales, 0) as TotalSales
    FROM(
        SELECT st, IFNULL(0.0, 0) as TotalSales
        FROM tZip
        GROUP BY st
    )
    LEFT JOIN SalesPerState USING (st)
),
LowestStateSale as(
    SELECT year_month, Sales
    FROM(
        SELECT year_month, Sales, NumOrders, NumCust, 
                (NumOrders / NumCust) as OrdersPerCust, 
                (Sales / NumCust) as SalesPerCust, 
                (Sales / NumOrders) as SalesPerOrder
        FROM(
            SELECT *, sum(tot) as Sales, count(DISTINCT(order_id)) as NumOrders, count(DISTINCT cust_id) as NumCust
            FROM(
                SELECT DISTINCT(date), *, (qty * unit_price) as tot
                FROM year_month
                JOIN tOrder USING (date)
                JOIN tOrderDetail USING (order_id)
                JOIN tProd USING (prod_id)
                GROUP BY order_id, prod_id
            )
            GROUP BY year_month
        )
    )
    ORDER BY Sales ASC
    LIMIT 1
)
SELECT cust_id, first, last, addr, city, st, zip, sum(unit_price * qty) as SalesSince
FROM(
    SELECT DISTINCT(date), * 
    FROM year_month
    JOIN tOrder USING (date)
    JOIN tOrderDetail USING (order_id)
    JOIN tCust USING (cust_id)
    JOIN tZip USING (zip)
    JOIN tProd USING (prod_id)
    WHERE st IN (SELECT st
                 FROM StatesSales
                 WHERE TotalSales = 0.00
                 )
    GROUP BY cust_id, order_id, year_month
    )
WHERE year_month > (SELECT year_month FROM LowestStateSale)
GROUP BY cust_id
ORDER BY cust_id ASC
;"""
s.RunQuery(sql4)

,cust_id,first,last,addr,city,st,zip,SalesSince
0,3,Dodonna,Garza,3639 Briarwood Court,Tarzan,TX,79783,28.09
1,6,Motti,Hunt,1919 Smith Street,Hamden,CT,06514,171.32
2,7,Mace Windu,Woodward,5655 Jefferson Court,Milan,NM,87021,136.59
3,11,Senator Ask Aak,Chen,2027 Pheasant Run,Wayne,NJ,07470,122.56
4,12,Jabba,Oneill,9371 Walnut Street,Swampscott,MA,01907,1624.61
...,...,...,...,...,...,...,...,...
113,297,Tey How,Rodriguez,9164 Madison Street,Brick,NJ,08723,26.82
114,301,Rabe,Mitchell,8021 Heather Lane,Pindall,AR,72669,112.51
115,302,Taun We,Miller,4011 Sycamore Drive,Yellow Jacket,CO,81335,222.16
116,305,Dodonna,Francis,8070 Briarwood Court,Buckland,AK,99727,113.88


---

5) Get a list of customers who did not purchase anything in the most recent month of data, along with their average sales for all months prior.

Return:

- customer id
- name, address, zip, city, st (abbreviation is fine)
- total sales for most recent month (to confirm they are all 0)
- average sales for all months prior

Order the results with the largest average monthly sales on top.

In [9]:
sql5 = """
WITH
RecentMonth as
(
    SELECT max(year_month) as recent_month
    FROM year_month
),
year_month as
(
SELECT date,
    substr(date, 1, 7) as year_month
FROM tOrder
),
NumMonths as
(
    SELECT count(DISTINCT year_month) as num_months
    FROM year_month
),
SalesRecentMonth as
(
    SELECT DISTINCT(date), *
    FROM year_month
    JOIN tOrder USING (date)
    JOIN tOrderDetail USING (order_id)
    JOIN tCust USING (cust_id)
    WHERE year_month IN RecentMonth
    GROUP BY cust_id
),
CustList as(
    SELECT cust_id, IFNULL(0.0, 0) as RecentSales
    FROM(
        SELECT DISTINCT(date), *
        FROM year_month
        JOIN tOrder USING (date)
        JOIN tOrderDetail USING (order_id)
        JOIN tCust USING (cust_id)
        JOIN tProd USING (prod_id)
        WHERE cust_id NOT IN (SELECT cust_id FROM SalesRecentMonth)
    )
    LEFT JOIN tCust USING (cust_id)
    GROUP BY cust_id
)
SELECT cust_id, first, last, addr, zip, city, RecentSales, sum(unit_price * qty) / (SELECT num_months-1 FROM NumMonths) as AvgSales
FROM(
    SELECT DISTINCT(date), cust_id, unit_price, qty
    FROM year_month
    JOIN tOrder USING (date)
    JOIN tOrderDetail USING (order_id)
    JOIN tCust USING (cust_id)
    JOIN tProd USING (prod_id)
    WHERE year_month NOT IN RecentMonth
)
JOIN tCust USING (cust_id)
JOIN tZip USING (zip)
JOIN CustList USING (cust_id)
WHERE cust_id IN (SELECT cust_id FROM CustList)
GROUP BY cust_id
ORDER BY AvgSales DESC
;"""
s.RunQuery(sql5)

,cust_id,first,last,addr,zip,city,RecentSales,AvgSales
0,39,Unkar Plutt,Woodward,5772 4th Street,51650,Riverton,0.0,590.661212
1,46,Captain Antilles,Walker,8516 Pheasant Run,20005,Washington,0.0,561.206667
2,152,Unkar Plutt,Schmidt,9546 Brookside Drive,13623,Chippewa Bay,0.0,533.290000
3,207,Jobal,Mitchell,1198 West Avenue,38629,Falkner,0.0,519.972424
4,202,Gold Leader,Elliott,9326 Sycamore Street,25938,Victor,0.0,509.008182
...,...,...,...,...,...,...,...,...
178,142,Mace Windu,Greene,9572 9th Street,61957,Windsor,0.0,43.359394
179,243,Darth Maul,Walters,7851 Magnolia Court,58046,Hope,0.0,39.101212
180,241,Bala-Tik,Zhang,1559 Lake Avenue,93673,Traver,0.0,36.436364
181,233,Clone Commander Cody,Benson,4836 Front Street,28390,Spring Lake,0.0,25.335152


---
6) Are there any products we haven't sold at least 1 of each month?

If so, return:
 
- prod_id
- prod_desc
- unit_price
- year_month

Order the results by prod_id, year_month, both ascending

In [10]:
sql6 = """
WITH year_month as
(
SELECT date,
    substr(date, 1, 7) as year_month
FROM tOrder
),
AllPossible as(
    SELECT DISTINCT(year_month), prod_id, prod_desc, unit_price
    FROM year_month
    JOIN tProd
)
SELECT *
FROM AllPossible
WHERE (year_month, prod_id) NOT IN (SELECT year_month, prod_id
                                FROM year_month
                                JOIN tOrder USING (date)
                                JOIN tOrderDetail USING (order_id)
                                JOIN tCust USING (cust_id)
                                JOIN tProd USING (prod_id)
                                GROUP BY prod_id, year_month
                                )
ORDER BY prod_id, year_month ASC
;"""
s.RunQuery(sql6)

,year_month,prod_id,prod_desc,unit_price
0,2019-09,300,Washer,0.10
1,2019-09,301,Nail,0.25
2,2019-09,305,Bradawl,1.99
3,2019-03,307,Sandpaper,3.00
4,2019-05,308,Screwdriver,3.00
5,2019-07,309,Chisel,4.99
6,2019-04,310,Scraper,7.99
7,2019-07,312,Plane,10.99
8,2019-06,313,Wrench,11.00
9,2019-06,314,Mallet,12.00


---
7) What are our top 5 selling products (in terms of total dollars sold)?

Return:

- prod_id
- prod_desc
- unit_price
- total sales (i.e. sum of qty * unit price)

Order the results by total sales, descending

In [11]:
sql7 = """
SELECT prod_id, prod_desc, unit_price, sum(unit_price * qty) as TotSales
FROM tOrderDetail
JOIN tProd USING (prod_id)
GROUP BY prod_id
ORDER BY TotSales DESC
LIMIT 5
;"""
s.RunQuery(sql7)

,prod_id,prod_desc,unit_price,TotSales
0,329,Chainsaw,499.99,943481.13
1,328,Workbench,300.00,633300.00
2,327,Ladder,80.00,154000.00
3,326,Drill,69.00,131652.00
4,325,Toolbox,50.00,100850.00


---

8) What month did we have our highest sales, and what was our top selling product that month? (All in terms of dollars).

Return:

- year_month (as a single column, with values like YYYY-MM)
- prod_id
- prod_desc
- The total sales for that month, for all products
- The total sales for that month, for the top selling product only
- The total quantity for that month, for the top selling product only

In [12]:
sql8 = """
WITH year_month as
(
SELECT date,
    substr(date, 1, 7) as year_month
FROM tOrder
),
HighestMonthProds as(
    SELECT DISTINCT(date), *
    FROM tOrderDetail
    JOIN tProd USING (prod_id)
    JOIN tOrder USING (order_id)
    JOIN year_month USING (date)
    WHERE year_month IN (
                    SELECT year_month
                    FROM(
                        SELECT *, sum(tot) as Sales, count(DISTINCT(order_id)) as NumOrders, count(DISTINCT cust_id) as NumCust
                        FROM(
                            SELECT DISTINCT(date), *, (qty * unit_price) as tot
                            FROM year_month
                            JOIN tOrder USING (date)
                            JOIN tOrderDetail USING (order_id)
                            JOIN tProd USING (prod_id)
                            GROUP BY order_id, prod_id
                        )
                        GROUP BY year_month
                    )
                    ORDER BY Sales DESC
                    LIMIT 1
            )
    GROUP BY order_id, prod_id
),
TotalSales as(
    SELECT sum(qty * unit_price) as MonthlyTot, year_month
    FROM HighestMonthProds
)
SELECT year_month, prod_id, prod_desc, MonthlyTot, sum(qty * unit_price) as SalesProd, sum(qty) as QuantProd
FROM HighestMonthProds
JOIN TotalSales USING(year_month)
GROUP BY prod_id
ORDER BY SalesProd DESC
LIMIT 1
;"""
s.RunQuery(sql8)

,year_month,prod_id,prod_desc,MonthlyTot,SalesProd,QuantProd
0,2021-10,329,Chainsaw,251768.12,86998.26,174


In [13]:
ag.Setup()

In [14]:
#File name for your submission
#Just use the assignment name/number
#Do not use underscores
fileName = "FinalProject.DATA302"

#Name of the database - must exist one directory up in the 'databases' folder
dbName = 'store.db'

#Dont change these:
PATH_SUB = 'submissions/' + fileName
PATH_DB = '../databases/' + dbName

#Make a dictionary to store your queries
mySql = dd(dict)

In [15]:
mySql['sql1']['sql'] = sql1

In [16]:
mySql['sql2']['sql'] = sql2

In [17]:
mySql['sql3']['sql'] = sql3

In [18]:
mySql['sql4']['sql'] = sql4

In [19]:
mySql['sql5']['sql'] = sql5

In [20]:
mySql['sql6']['sql'] = sql6

In [21]:
mySql['sql7']['sql'] = sql7

In [22]:
mySql['sql8']['sql'] = sql8

In [23]:
with open(PATH_SUB, 'w') as f:
    json.dump(mySql, f)

In [24]:
st = ag.Student(PATH_SUB, dbName)

In [25]:
st.CheckMySubmission()

Output saved to: output/me_2023-05-14-15-17-52-707791.DATA3024u 

===== RUNNING SETUP QUERIES =====

Note that the student version will run all action queries, however

some actions are disallowed during grading. See the documentation for more info.





===== DONE WITH SETUP QUERIES =====







-------------Submission for sql1 found-------------



WITH year_month as

(

SELECT date,

    substr(date, 1, 7) as year_month

FROM tOrder

)

SELECT year_month, Sales, NumOrders, NumCust, 

        (NumOrders / (NumCust * 1.0)) as OrdersPerCust, 

        (Sales / NumCust) as SalesPerCust, 

        (Sales / NumOrders) as SalesPerOrder

FROM(

    SELECT *, sum(tot) as Sales, count(DISTINCT(order_id)) as NumOrders, count(DISTINCT cust_id) as NumCust

    FROM(

        SELECT DISTINCT(date), *, (qty * unit_price) as tot

        FROM year_month

        JOIN tOrder USING (date)

        JOIN tOrderDetail USING (order_id)

        JOIN tProd USING (prod_id)

        GROUP BY order_id, prod_